# 1- Packages Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# 2- Approach

In this notebook, we perform elementary preprocessing operations on our 2 data files: <br/>
- Masterfile 24h <br/>
- Masterfile 6h <br/>
- Masterfile 4h <br/>
- Masterfile 2h <br/>

# 3- Data Import

In [11]:
creatinine=True
path_to_add='master_file_different_hours/'
if creatinine:
    path_to_add='Creatinine_data_master/'

data24 = pd.read_csv('../data/'+path_to_add+'Masterfile24h.csv', index_col=0)
data6 = pd.read_csv('../data/'+path_to_add+'Masterfile6h.csv', index_col=0)
data4 = pd.read_csv('../data/'+path_to_add+'Masterfile4h.csv', index_col=0)
data2 = pd.read_csv('../data/'+path_to_add+'Masterfile2h.csv', index_col=0)

In [10]:
data2hb = data2

In [12]:
data2cr = data2

In [17]:
data2hb = data2hb.loc[data2hb["Hem_base_timelag"]>=-30]

In [21]:
data2cr = data2cr.loc[data2cr["Cr_base_timelag"]>=-30]

In [16]:
data2cr.shape

(4786, 85)

In [22]:
data2hb.loc[data2hb.subject_id.isin(data2cr.subject_id)].shape

(4331, 83)

In [80]:
import re
def get_race(race):
    return re.split(r'[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>? ]', race)[0]
    
def convert_race(race):
    if race in ['WHITE', 'BLACK', 'HISPANIC', 'ASIAN']:
        return race
    return 'OTHER'

def get_race_straight(race_long):
    race= re.split(r'[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>? ]', race_long)[0]
    if race in ['WHITE', 'BLACK', 'HISPANIC', 'ASIAN']:
        return race
    return 'OTHER'


In [3]:
patients4 = data4.subject_id.unique()
patients2 = data2.subject_id.unique()

print(data2.shape)
print(data4.shape)
#print(len(inter))
data4 = data4.loc[data4.subject_id.isin(patients2)]
data6 = data6.loc[data6.subject_id.isin(patients2)]
data24 = data24.loc[data24.subject_id.isin(patients2)]

(6281, 83)
(6593, 83)


In [5]:
data ={}

In [6]:
data[24]= data24
data[6]= data6
data[4]= data4
data[2]= data2

# 4- Elementary Preprocessing

### 4.1- Trend Features?

### 4.2- Timelag filter

Here, we delete rows with a timelag greater than 30 days.

In [7]:
nb_max_days = 30

In [8]:
for i in [2,4,6,24]:   
    print("Number of rows deleted for "+str(i) +"hours : ", len(data[i].loc[data[i]["Hem_base_timelag"]<-30]))
    data[i] = data[i].loc[data[i]["Hem_base_timelag"]>=-30]

Number of rows deleted for 2hours :  142
Number of rows deleted for 4hours :  142
Number of rows deleted for 6hours :  142
Number of rows deleted for 24hours :  142


In [9]:
6281-142

6139

### 4.3- Features selection

Here, we delete features using: <br/>
- our correlation analysis <br/>
- the medical insights about redundant features

We do not perform any "best subset selection" here.

In [58]:
id_features_tbr = ['subject_id', 'icustay_id', 'hadm_id']
time_features_tbr = ['Hb_baseline_time','admittime', 'Hb_initial_time']
corr_features_tbr = ['Hematocrit', 'PT','MCH', 'MCV']
corr_features_tbr += [c for c in data24.columns if ((c.startswith("MAP")))]
other_features_tbr = ['icu_length_of_stay', 'admission_location', 'diagnosis']
features_to_be_removed = id_features_tbr + time_features_tbr + corr_features_tbr + other_features_tbr

In [59]:
for i in [2,4,6,24]:   
    data[i] = data[i].drop(columns = features_to_be_removed, axis=1)


### 4.4- Missing Values Filter

In [60]:
df_missing = {}

In [61]:
for i in [2,4,6,24]:   
    df_missing[i] = pd.DataFrame({"column":list(data[i].columns), "% missing":[round(sum(data[i][c].isnull())/len(data[i])*100, 2) for c in data[i].columns]})

In [62]:
i=2
df_missing[i].loc[df_missing[i]['% missing']>=80]

,% missing,column
15,96.24,Ferritin
16,98.01,Folate
18,95.76,Iron
19,95.86,"Iron Binding Capacity, Total"
29,96.71,"Protein, Total"
33,95.86,Transferrin
35,97.41,Vitamin B12
65,97.46,vassopresor_doses


In [63]:
i=24
features_to_drop = list(df_missing[i].loc[df_missing[i]['% missing']>=80]["column"].unique())
features_to_drop.remove("vassopresor_doses")
features_to_drop = [c for c in features_to_drop if c in data[i].columns]

In [64]:
for i in [2,4,6,24]:   
    data[i] = data[i].drop(columns = features_to_drop, axis=1)

### 4.5- Encode categorical variables

In [65]:
for i in [2,4,6,24]:   
    for col in [ 'admission_type', 'current_service', 'Bleed_dx']:
        le = preprocessing.LabelEncoder()
        le.fit(data[i][col])
        data[i][col]=le.transform(data[i][col])
    

In [66]:
for i in [2,4,6,24]:   
    data[i]['vassopresor_doses'] = data[i]['vassopresor_doses'].fillna(0)


In [69]:
def is_female(g):
    if g=='F':
        return 1
    return 0

In [71]:
for i in [2,4,6,24]:   
    data[i]['is_female'] = data[i]['gender'].apply(is_female)
    data[i].drop(columns='gender',inplace=True)



In [72]:
for i in [2,4,6,24]:   
    data[i]['ethnicity'] = data[i]['ethnicity'].apply(get_race_straight)




# 5- Data Export

In [73]:
print(data[i].shape)
print(data[i].columns)

(6139, 59)
Index(['Hb_value_baseline', 'Hem_base_timelag', 'Hb_value_initial',
       'ageatadmin', 'admission_type', 'ethnicity', 'current_service',
       'Bleed_dx', 'Albumin', 'Bicarbonate', 'Bilirubin, Total',
       'Calcium, Total', 'Chloride', 'Creatinine', 'INR(PT)', 'Lactate',
       'Lactate Dehydrogenase (LD)', 'MCHC', 'pCO2', 'pH', 'Phosphate',
       'Platelet Count', 'pO2', 'Potassium', 'PTT', 'Red Blood Cells',
       'Sodium', 'Urea Nitrogen', 'White Blood Cells', 'DBP_max', 'DBP_mean',
       'DBP_min', 'DBP_sd', 'GLU_max', 'GLU_mean', 'GLU_min', 'GLU_sd',
       'HR_max', 'HR_mean', 'HR_min', 'HR_sd', 'RR_max', 'RR_mean', 'RR_min',
       'RR_sd', 'SBP_max', 'SBP_mean', 'SBP_min', 'SBP_sd', 'SPO2_max',
       'SPO2_mean', 'SPO2_min', 'SPO2_sd', 'TEMP_max', 'TEMP_mean', 'TEMP_min',
       'TEMP_sd', 'vassopresor_doses', 'is_female'],
      dtype='object')


In [74]:
for i in [2,4,6,24]:
    export_name_data = '../data/master_file_different_hours/PreProcessed_Masterfile{}h.csv'.format(str(i))
    data[i].to_csv(export_name_data)